<h1 style="
    font-family: Garamond;
    font-size: 34px;
    color: #2774AE;
    font-weight: 700;
    text-shadow:
        -1px -1px 0 #000,
         1px -1px 0 #000,
        -1px  1px 0 #000,
         1px  1px 0 #000;
">
Assignment 2A <em>
</h1>

<h2 style="
    font-family: Garamond;
    font-size: 22px;
    color: #FFD100;
    font-style: italic;
    text-shadow:
        -1px -1px 0 #000,
         1px -1px 0 #000,
        -1px  1px 0 #000,
         1px  1px 0 #000;
">
Econ 409 & Econ 442B
</h2>

<h3 style="
    font-family: Garamond;
    font-size: 20px;
    color: #2774AE;
    font-style: italic;
">
Jacob Williams, Praus Paek, Joshua Kenworthy, Agnibha Bhattacharya, and Ignacio Ramirez
</h3>

<h3 style="
    font-family: Garamond;
    font-size: 20px;
    color: #000000;
    font-style: italic;
">
University of California, Los Angeles
</h3>

<h3 style="
    font-family: Garamond;
    font-size: 20px;
    color: #000000;
    font-style: italic;
">
February 3, 2026
</h3>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wooldridge as woo
import scipy.stats as stats
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import seaborn as sns
import statsmodels.api as sm
from linearmodels import IV2SLS

### Note to reader:



<h2 style="
    font-family: Garamond;
    font-size: 22px;
    color: #FFD100;
    font-style: italic;
    text-shadow:
        -1px -1px 0 #000,
         1px -1px 0 #000,
        -1px  1px 0 #000,
         1px  1px 0 #000;
">
Theoretical Questions
</h2>

---

### Question 1: State-Space Model
Consider the state space model presented in the lecture:
$$y_{t}=x_{t}+v_{t}$$
$$x_{t}=ax_{t-1}+w_{t}$$
$$|a|<1$$
With $v_{t}\sim N(0,\sigma_{v}^{2})$, $w_{t}\sim N(0,\sigma_{w}^{2})$, and $v_{t}$, $w_{t}$ are independent white noise.

**a.** Write down the recursive formula for the filtered estimate $\hat{x_{t}}$ using a constant gain $k\in[0,1]$.

**b.** Explain intuitively what happens to the filtered series and the resulting multi-step forecasts $\overline{y_{t+n}}=a^{n}\hat{x_{t}}$ when $k$ is very close to 0 versus when $k$ is very close to 1.

**c.** State the formula for the optimal Kalman gain $k^{Kalman}$ and briefly explain when it lies between 0 and 1.


### Answers:


**a. Recursive formula for the filtered estimate $\hat{x_{t}}$:**

The filtered estimate using a constant gain $k$ is:
$$\hat{x_{t}} = a\hat{x_{t-1}} + k(y_{t} - a\hat{x_{t-1}})$$

**b. Intuitive explanation of gain $k$:**

* **When $k \to 0$:** The model ignores new observations ($y_t$) and relies almost entirely on the transition equation (the previous state). The filtered series will be very smooth, but the multi-step forecasts $\overline{y_{t+n}} = a^{n}\hat{x_{t}}$ will be slow to react to structural shifts in the data.
* **When $k \to 1$:** The model puts maximum weight on the most recent observation, assuming $y_t$ is a very accurate reflection of the state. This makes the filtered series "noisy" or "jagged," and forecasts will be highly sensitive to temporary shocks or measurement errors in the most recent data point.

**c. Optimal Kalman gain $k^{Kalman}$**

The optimal Kalman gain $k^{Kalman}$ (often denoted as $k^*$) for this state-space model is given by the ratio of the predicted state variance to the total variance (state variance plus observation noise):

$$k^* = \frac{a^2 \sigma^2 + \sigma_w^2}{a^2 \sigma^2 + \sigma_w^2 + \sigma_v^2}$$

**Explanation:**

* **Range between 0 and 1:** The gain lies in the interval $[0, 1]$ because all components ($\sigma_w^2$, $\sigma_v^2$, and $a^2 \sigma^2$) are variances or squared terms, making them non-negative. Since the denominator is the sum of the numerator and the positive observation noise $\sigma_v^2$, the denominator is always greater than or equal to the numerator, forcing the value to be $\leq 1$.
* **Economic Intuition:** * If observation noise ($\sigma_v^2$) is very large relative to state uncertainty, $k^*$ approaches 0, meaning we trust our previous model more than the new data.
    * If process noise ($\sigma_w^2$) or state uncertainty is very high relative to observation noise, $k^*$ approaches 1, meaning we trust the new data more to update our estimate.

---

### Question 2: The Sharpe Ratio
**a.** Define the Sharpe ratio and carefully explain what it measures in economic terms. Why is the risk-free rate subtracted in the numerator?

**b.** Explain the pitfall of the Sharpe ratio and discuss why it can make the Sharpe ratio misleading when comparing strategies or assets.

**c.** Why is it common practice to convert annual risk free rates to the same frequency as the portfolio returns (e.g., divide by 12 for monthly) before computing monthly excess returns and the Sharpe ratio?

### Answers:

**a. Definition and Economic Measurement:**

The Sharpe Ratio is defined as:
$$S = \frac{R_i - R_f}{\sigma_i}$$
It measures the **excess return per unit of total risk** (volatility). The risk free rate is subtracted in the numerator to isolate the "risk premium"—the compensation an investor receives specifically for taking on the uncertainty of the asset rather than staying in a guaranteed investment.

**b. Pitfalls and Misleading Comparisons:**

A major pitfall is that the Sharpe ratio assumes returns are **normally distributed**. It does not distinguish between "good" upside volatility and "bad" downside volatility. An asset with high positive skewness (frequent small losses but occasional huge gains) might have a lower Sharpe ratio than a "steady" asset that actually carries significant hidden "tail risk" (the risk of rare, catastrophic crashes).

**c. Frequency Conversion:**

It is common practice to convert rates to the same frequency to ensure the **excess return calculation is mathematically consistent**. If you subtract an annual $R_f$ (e.g., 4%) from a monthly return (e.g., 1%), you would erroneously calculate a massive negative return. Dividing by 12 aligns the time-horizon of the "reward" with the time-horizon of the "risk" measured in that period.

---

### Question 3: Consider the CAPM Model
$$R_{i}(t)=\alpha_{i}+\beta_{i}R_{M}(t)+e_{i}(t)$$
Where $R_{i}(t)$ and $R_{M}(t)$ are excess returns (over the risk-free rate) of the strategy/asset $i$ and the market proxy, respectively, and $E[e_{i}(t)]=0$.

**a.** Provide a precise economic interpretation of $\alpha_{i}$ and $\beta_{i}$ in the context of evaluating an active investment strategy.

**b.** Explain why a negative beta combined with positive alpha is particularly attractive to investors, especially during periods of market stress.

**c.** Why might a strategy with a very high positive alpha but a beta close to 1.0 still underperform a passive market index during a prolonged bull market, even if the alpha is statistically significant?

### Answers:

**a. Economic Interpretation of $α{i}$ and $β{i}$**

**Alpha ($α{i}$)**: This represents the strategy's excess return when the market's excess return ($R_{M}$) is zero. Economically, it measures the "abnormal" rate of return—the value added by the portfolio manager's skill (e.g., stock selection or market timing) that cannot be explained by the strategy's exposure to the broad market. A positive, statistically significant alpha implies the strategy is outperforming the risk-adjusted benchmark.

**Beta ($β{i}$)**: This measures the sensitivity of the strategy's returns to market movements. A beta of 1.0 implies the strategy moves in lockstep with the market (e.g., the S&P 500). A beta greater than 1 indicates the strategy is more volatile (amplified moves) than the market, while a beta less than 1 indicates it is less volatile. In active management, beta essentially quantifies the amount of systematic market risk the strategy is taking.

**b. The Attraction of Negative Beta and Positive Alpha**

A strategy combining negative beta (β < 0) with positive alpha (α > 0) is highly attractive, particularly during market stress, because it acts as a profit-generating hedge.

**Negative Beta**: A negative beta implies that when the market performs poorly (negative RM), the strategy tends to perform well. This creates a negative correlation with the broader portfolio, providing diversification benefits and dampening overall portfolio volatility during crashes.

**Positive Alpha**: Unlike a simple short position (which has negative beta but typically negative expected returns over the long run due to the equity risk premium), a positive alpha ensures the strategy generates excess returns over time, independent of market direction.

Investors seek this profile because it offers "insurance" that pays you to hold it, rather than costing a premium.

**c. Underperformance in a Bull Market (High Alpha, Beta ≈ 1)**

Even with a high positive α, a strategy can underperform a passive index during a prolonged, aggressive bull market if its β is "close to" but strictly less than 1.0.
Mathematically, the difference between the strategy return ($R{i}$) and the market return ($R_{M}$) is: $$R{i} − R_{M} = α + (β−1)R_{M}$$
​
If the market return ($R_{M}$) is extremely high (a strong bull market), and β < 1 (e.g., 0.8 or 0.9), the term $(β−1)R_{M}$ becomes a large negative number. If this negative drag from not being fully exposed to the market exceeds the value of α, the strategy will lag the benchmark.

Additionally, standard CAPM assumes idiosyncratic risk $e_{i}(t)$ averages to zero. In a specific bull run driven by narrow market concentration, a strategy that is diversified (lower beta) or underweight those specific mega-cap drivers might experience negative residuals ($e_{i}$ < 0) temporarily, causing underperformance despite a theoretical long-term structural alpha.


​



---

### Question 4: Skewness and Kurtosis
Consider the definitions of skewness and kurtosis presented in the lecture:
$$skew=\frac{1}{T-1}\sum_{t=1}^{T}\left(\frac{R_{t}-\overline{R}}{\hat{\sigma}}\right)^{3}$$
$$kurtosis=\frac{1}{T-1}\sum_{t=1}^{T}\left(\frac{R_{t}-\overline{R}}{\hat{\sigma}}\right)^{4}-3$$

**a.** Provide the economic interpretation of a large negative skewness in monthly excess returns of a portfolio or trading strategy. What does it imply about the distribution of outcomes an investor should expect?

**b.** Explain why, under the assumption of normally distributed returns (skew = 0, kurtosis = 0), the mean and standard deviation are sufficient statistics for describing risk and expected performance, but this breaks down when skewness and/or excess kurtosis are materially nonzero.

### Answers:

**a. Economic Interpretation of Large Negative Skewness**

Skewness measures the asymmetry of return distribution around the mean. A large negative skewness implies that the "tail" on the left side of the distribution is longer or fatter than the right side. Economically, this indicates a high frequency of small positive gains disrupted by rare but massive losses. This is often described as "crash risk" or "tail risk." An investor should expect a return profile similar to selling insurance: steady, consistent income most of the time, punctuated by catastrophic drawdowns.

**Example**: In our lectures (3.1), the "Low Variance Series with Rare Crash" exhibited a large negative skew (e.g., -12.47) due to a single catastrophic -68% day, despite having a high Sharpe ratio prior to the crash.

**b. Breakdown of Mean-Variance Sufficiency**

Under the assumption of a Normal Distribution, the distribution is perfectly symmetric (Skew = 0) and has a defined tail thickness (Kurtosis = 0 relative to Normal). In this specific example, the mean (expected return) and standard deviation (volatility) are sufficient statistics — they contain 100% of the information needed to describe the probabilities of all future outcomes.

This breaks down when moments are nonzero because if a distribution has negative skew or high kurtosis (fat tails), extreme events happen far more frequently than a Normal distribution predicts. Standard deviation treats upside volatility and downside volatility equally, but investors care significantly more about downside crashes. A strategy can also have a low standard deviation (low volatility) and a high mean (high Sharpe ratio), yet possess massive negative skewness (e.g., a strategy that bets against rare events). 

Relying solely on mean and standard deviation would lead an investor to drastically underestimate the probability of a catastrophic loss. To capture this, we must look at metrics like the Sortino Ratio (which isolates downside deviation) or Maximum Drawdown.



---

### Question 5: US Treasury Bonds
A 30-year US Treasury bond has the following characteristics:
* Face value (par value): $1,000
* Annual coupon rate: 4% (paid semiannually)
* Current market price: $950 (clean price, quoted per $1,000 face value)

**(a)** Write the bond pricing equation that must be solved for the semiannual yield to maturity $i$ (the constant semiannual discount rate that equates the present value of all promised cash flows to the current market price of $950). Specify the number of periods $N$ and the size of each coupon payment.

**(b)** If the bond were trading at par ($1,000) instead of $950, what would the semiannual YTM be? What does this tell you about the relationship between coupon rate, market price, and YTM?

### Answers



---

### Question 6: Futures and Forwards
Futures contracts are standardized versions of forward contracts that address the two main limitations of forwards: illiquidity and counterparty risk. Explain clearly what these two limitations mean in the context of forward contracts and why they pose practical problems for market participants (e.g., hedgers or speculators).

### Answers



---

### Question 7: The Taylor Rule
In the standard Taylor rule:
$$i_{t}=r^{*}+\pi_{t}+0.5(\pi_{t}-\pi^{*})+0.5(y_{t}-\overline{y_{t}})$$

**a.** Explain clearly what the coefficients 0.5 (on inflation) and 0.5 (on output gap) imply about the Federal Reserve's relative concern for inflation versus output stabilization.

**b.** If the actual federal funds rate is persistently below the rate prescribed by the Taylor rule for a long period, how is monetary policy conventionally described? What are the main arguments that central banks give to justify such a situation?

### Answers



---

### Question 8: FOMC Statements
Find all the text changes in the text of FOMC statements for the following dates:
* September 18, 2024
* December 18, 2024
* January 29, 2025

(Refer to: https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm)

Explain how those changes would affect expectations of feds funds rate over the next 12 months.

### Answers

---
---
---

<h2 style="
    font-family: Garamond;
    font-size: 22px;
    color: #FFD100;
    font-style: italic;
    text-shadow:
        -1px -1px 0 #000,
         1px -1px 0 #000,
        -1px  1px 0 #000,
         1px  1px 0 #000;
">
Coding and Bloomberg Questions
</h2>

### Question 2: Data Exploration (CFNAI)
The `hw2_data.csv` contains columns for SHY (a 1-3-year bond ETF), PCE inflation (%), the CFNAI index, and the fed funds rate. 

The Chicago Fed National Activity index (CFNAI) is described as: *"A zero value for the index indicates that the national economy is expanding at its historical trend rate of growth; negative values indicate below-average growth; and positive values indicate above-average growth."* For later questions, we will be using this as our "output gap" variable.

**a.** Plot the CFNAI variable.

**b.** Think about how we used trend to construct the output gap using industrial production in class. Why might it make sense to use the CFNAI as a proxy for the output gap?

### Question 1: Bloomberg Terminal - PCE Inflation
Log into the Bloomberg Terminal and search for **PCE CYOY Index**. Plot the data from 01/01/1959 - 11/30/2025. Plot the Actual Economic Release Values on the Series Release Date.

**a.** When was the last data point released?

**b.** What was the reference month (the month for which inflation was being measured)?

**c.** Turn off the series release date setting. When is it indexing the last release now?

**d.** Describe how changes in the dates you use can affect the forecasts you make.

---

---

### Question 3: Predicting the Fed Funds Rate
**a.** Calculate the inflation gap, and assume we are using a target rate of inflation of 2%.

**b.** Fit the following Taylor rule model over a rolling 3-year window and use it to make a prediction for the next out of sample fed funds rate:
$$fedfunds_{t}=\alpha+\beta_{1}CFNAI_{t}+\beta_{2}(\pi_{t}-2)+v_{t}$$

**c.** Plot the predicted fed funds rate against the observed rate.

---

### Question 4: Trading Strategy
We will build a simple trading strategy that goes long on the SHY when the predicted fed funds rate is higher than the observed rate and short when the predicted fed funds rate is lower than the observed rate. Note that when you get a signal that's either 1 or -1, you hold it only for one time period.

**a.** Generate a signal column that corresponds to the strategy described above.

**b.** Plot the strategy cumulative returns.

**c.** Calculate the Annualized Return.

**d.** Calculate the skewness of the returns. What does this tell you about your strategy?

---